# ДЗ №3. Ансамбли (50 баллов)

В этом домашнем задании нужно реализовать алгоритм Random Forest и реализовать методы ансамблирования Stacking и Blending.

## Данные

Вы будете работать с [датасетом Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic/data).
* Если у вас нет возможности зарегестрироваться на Kaggle, то данные лежат [тут](https://drive.google.com/file/d/16SJ4FeqMIsfzqpbcm82Yl8F6MU7MODr9/view?usp=drive_link).

* _Если вы используете данные с диска, то сами сформируйте train/val/test выборку_


Предобработайте данные как мы делали в папке [Работа с признаками](https://github.com/runnerup96/SBT-machine-learning-seminars/tree/main/Работа%20с%20признаками) - заполните пропущенные данные, обработайте редкие значения, закодируйте категориальные признаки, создайте и отберите признаки.

## Задание №1 Random Forest (20 баллов)

Реализуйте __RandomForest__ алгоритм. В качестве базового алгоритма возьмите [DecisionTreeСlassifer](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) из sklearn.

* Сравните качество с [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) из sklearn на датасете SpaceShipTitanic. Различие должно быть не более **5%** по метрике **Accuracy**.

* Подберите гиперпараметры для вашего алгоритма с помощью __Optuna__ (пример с семинара [тут](https://github.com/runnerup96/SBT-machine-learning-seminars/blob/main/Деревья%20и%20ансамбли/Деревья%20решений%20и%20ансамбли.ipynb)) и продемонстрируйте качество на тестовой выборке.

**Если вы смогли участвовать в Kaggle соревновании, приложите скриншот в ноутбук вашего сабмита с этим алгоритмом**

In [1]:

! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00


In [2]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
class RandomForest:
    def __init__(
        self,
        n_estimators=100,
        criterion='gini',
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True,
    ):
        self.n_estimators = n_estimators
        self.bootstrap = bootstrap
        self.estimators = [
            DecisionTreeClassifier(
                criterion=criterion,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf
            )
            for _ in range(n_estimators)
        ]
    @staticmethod
    def bootstrapping(X, y, n_samples, size_samples):
      bootstrap_samples = []
      N = X.shape[0]
      for i in range(n_samples):
        idx = np.random.choice(N, size_samples, replace=True)
        X1 = X[idx]
        y1 = y[idx]
        bootstrap_samples.append([X1, y1])
      return bootstrap_samples

    def fit(self, X, y):
        """функция обучения модели"""
        # CODE HERE
        N = X.shape[0]
        samples = self.bootstrapping(X, y, self.n_estimators, int(N/2))
        for i,data in enumerate(samples):
          X1, y1 = data
          self.estimators[i].fit(X1, y1)

    def predict(self, X):
        """функция предсказания"""
        # CODE HERE
        return self.predict_proba(X) < 0.5

    def predict_proba(self, X):
        """функция предсказания вероятностей"""
        # CODE HERE
        N = X.shape[0]
        predictions = np.zeros((self.n_estimators, N))
        for i in range(self.n_estimators):
            pred_i = self.estimators[i].predict_proba(X)
            predictions[i] = pred_i[:,0]
        return np.mean(predictions, axis=0)

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
#Проверка количества и процента пропущенных значений для каждого признака
result = pd.concat([train.isnull().sum(),train.isnull().mean()],axis=1)
result.rename(index=str,columns={0:'total missing',1:'proportion'})

,total missing,proportion
PassengerId,0,0.000000
HomePlanet,201,0.023122
CryoSleep,217,0.024963
Cabin,199,0.022892
Destination,182,0.020936
Age,179,0.020591
VIP,203,0.023352
RoomService,181,0.020821
FoodCourt,183,0.021051
ShoppingMall,208,0.023927


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [8]:
le = LabelEncoder()
#будем считать, что по признакам ниже False у тех пассажиров, по которых информация неизвестна
train["CryoSleep"] = train["CryoSleep"].fillna(False).astype(int)
train["VIP"] = train['VIP'].fillna(False).astype(int)
train["Transported"] = train["Transported"].astype(int)

train["HomePlanet"] = le.fit_transform(train["HomePlanet"])
train['Destination'] = le.fit_transform(train['Destination'])

#создали новые 3 признака для Cabin с мыслью, что deck и side более информативны
#вряд ли от имени пассажира что то зависит
#Значит дропнем name и cabin
train['Cabin'].fillna('NaN/9999/NaN', inplace=True)
train['Cabin_deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
train['Cabin_side'] = train['Cabin'].apply(lambda x: x.split('/')[2])
train['Cabin_side'] = le.fit_transform(train['Cabin_side'])
train['Cabin_deck'] = le.fit_transform(train['Cabin_deck'])
train = train.drop(columns=["Cabin", "Name"])

In [9]:
#то же самое сделаем для test
test["CryoSleep"] = test["CryoSleep"].fillna(False).astype(int)
test["VIP"] = test['VIP'].fillna(False).astype(int)
test["HomePlanet"] = le.fit_transform(test["HomePlanet"])
test['Destination'] = le.fit_transform(test['Destination'])
test['Cabin'].fillna('NaN/9999/NaN', inplace=True)
test['Cabin_deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
test['Cabin_side'] = test['Cabin'].apply(lambda x: x.split('/')[2])
test['Cabin_side'] = le.fit_transform(test['Cabin_side'])
test['Cabin_deck'] = le.fit_transform(test['Cabin_deck'])
test = test.drop(columns=["Cabin", "Name"])

In [10]:
#Есть мысль, что если пассажир VIP, то расходы на RoomService	FoodCourt	ShoppingMall	Spa будут выше, чем у обычных пассажиров, тогда заменим пропущенные значения в данных столбцах
#средним значениям по этим показателям для каждого гостя отдельно
NA_col = ['RoomService','FoodCourt','ShoppingMall','Spa', 'VRDeck']
for i in NA_col:
  mean_vip = train.loc[train['VIP'] == 1, i].mean()
  mean_nonvip = train.loc[train['VIP'] == 1, i].mean()
  train.loc[(train['VIP'] == 0) & (train[i].isna()), i] = mean_nonvip
  train.loc[(train['VIP'] == 1) & (train[i].isna()), i] = mean_nonvip


In [11]:
NA_col = ['RoomService','FoodCourt','ShoppingMall','Spa', 'VRDeck']
for i in NA_col:
  mean_vip = test.loc[test['VIP'] == 1, i].mean()
  mean_nonvip = test.loc[test['VIP'] == 1, i].mean()
  test.loc[(test['VIP'] == 0) & (test[i].isna()), i] = mean_nonvip
  test.loc[(test['VIP'] == 1) & (test[i].isna()), i] = mean_nonvip

In [12]:
#с оставшимися признаками не знаю, что делать, так что дропнем их
train.dropna(inplace=True)

In [13]:
X, y = np.array(train.drop(columns=["Transported"])), np.array(train["Transported"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
rf = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=10)
rf.fit(X_train, y_train)
print("Sklearn implementaiton", accuracy_score(rf.predict(X_test), y_test))

Sklearn implementaiton 0.7985907222548444


In [15]:
clf = RandomForest(n_estimators=200, criterion='gini', max_depth=10)
clf.fit(X_train, y_train)
print("Our implemetation", accuracy_score(clf.predict(X_test), y_test))

Our implemetation 0.7921315325895478


In [16]:
#!pip install optuna
import optuna
optuna.logging.set_verbosity(0)

In [17]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }

    clf = RandomForest(
        **param
    )
    clf.fit(X_train, y_train)
    a = accuracy_score(clf.predict(X_test), y_test)

    return a

In [ ]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, study_name="RandomForest", direction="maximize")
study.optimize(objective, n_trials=150)

In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_params.items():
    print("    {}: {}".format(key, value))

In [ ]:
clf_opt = RandomForest(**study.best_params)
clf_opt.fit(X_train, y_train)

print("accuracy после подбора гиперпараметров:", accuracy_score(clf_opt.predict(X_test), y_test))

## Задание №2 Stacking (15 баллов)

Реализуйте стекинг(методом, представленном на лекции) над 3-5 алгоритмами (сами алгоритмы реализовывать не надо, брать готовые реализации). Продемонстрируйте качество вашей реализации: покажите метрики базовых моделей и ансамбля на тестовой выборке. Если в качестве метамодели вы используете логрег, выведите веса признаков каждой модели. Подбор гиперпараметров для ваших алгоритмов на ваше усмотрение.

**Если вы смогли участвовать в Kaggle соревновании, приложите скриншот вашего сабмита с этим алгоритмом**

In [ ]:
class Stacking:
    def __init__(self, estimators, final_estimator):
        """
        estimators : list
            Список базовых моделей
        final_estimator
            Метамодель для финального предсказания
        """
        self.estimators = estimators
        self.final_estimator = final_estimator

    def fit(self, X, y):
        """функция обучения модели"""
        # CODE HERE

    def predict(self, X):
        """функция предсказания"""
        # CODE HERE
        pass

    def predict_proba(self, X):
        """функция предсказания вероятностей"""
        # CODE HERE
        pass

## Задание №3 Blending (15 баллов)

Реализуйте блендинг(методом, представленном на лекции) с несколькими алгоритмами на ваш выбор. Продемонстрируйте качество вашей реализации: покажите метрики базовых моделей и ансамбля на тестовой выборке. Если в качестве метамодели вы используете логрег, выведите веса. Подбор гиперпараметров для ваших алгоритмов на ваше усмотрение.

**Если вы смогли участвовать в Kaggle соревновании, приложите скриншот вашего сабмита с этим алгоритмом**

In [ ]:
class Blending:
    def __init__(self, estimators, final_estimator):
        """
        estimators : list
            Список базовых моделей
        final_estimator:
            Метамодель для финального предсказания
        """
        self.estimators = estimators
        self.final_estimator = final_estimator

    def fit(self, X, y):
        """функция обучения модели"""
        # CODE HERE

    def predict(self, X):
        """функция предсказания"""
        # CODE HERE
        pass

    def predict_proba(self, X):
        """функция предсказания вероятностей"""
        # CODE HERE
        pass